# Gibbs Sampler Example 2: Pumps

Adapted from [Codes for Unit 5: pumpsmc.m](https://www2.isye.gatech.edu/isye6420/supporting.html).

Associated lecture video: [Unit 5 Lesson 13](https://www.youtube.com/watch?v=DKlP93sD0O0&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=49).